# Phase 1 - Ingestion and Cleaning

In the Phase 2 of the Case Study, we will carry out the following steps:
  - Ingest raw downloaded data
  - Output a combined dataset ready for analysis and modeling

In [4]:
import pandas as pd
import os
from sys import platform
import matplotlib.pyplot as plt
import datetime
import numpy as np
import pickle
import seaborn

In [5]:
# A helper function that you'll be using while reading the raw files
def is_integer(x):
    '''
    This function returns True if x is an integer, and False otherwise
    '''
    try:
        return (int(x) == float(x))
    except:
        return False

## Parameters

In [6]:
# Define the directories that contain the files downloaded
dir_cs = os.getcwd() + "/1805_download" # path to the directory where all the *.csv.zip files are located

# Define the output path for the pickle
pickle_file =  "./clean_data.pickle" # path to save cleaned data

In [32]:
# Identify the columns we'll be keeping from the dataset
cols_to_pick = ['id', 'loan_amnt', 'funded_amnt', 'term', 'int_rate', 'installment', 'grade', 'emp_length',
'home_ownership', 'annual_inc', 'verification_status', 'issue_d', 'loan_status',
'purpose', 'dti', 'delinq_2yrs', 'earliest_cr_line', 'open_acc', 'pub_rec',
'fico_range_high', 'fico_range_low', 'revol_bal', 'revol_util', 'total_pymnt',
'last_pymnt_d', 'recoveries'] # list of features to use for this study as indicated in the handout

# Identify the type of each of these column based on your CS-Phase 1 response
float_cols = ['loan_amnt', 'funded_amnt', 'installment','annual_inc','dti','delinq_2yrs','open_acc','pub_rec',
              'fico_range_high', 'fico_range_low','revol_bal','total_pymnt','recoveries']
cat_cols = ['term','grade', 'emp_length','home_ownership','verification_status', 'loan_status', 'purpose', 'earliest_cr_line'] # categorical features
perc_cols = ['int_rate', 'revol_util']
date_cols = ['issue_d', 'earliest_cr_line', 'last_pymnt_d']

# Ensure that we have types for every column
assert set(cols_to_pick) - set(float_cols) - set(cat_cols) - set(perc_cols) - set(date_cols) == set(["id"])

In [33]:
# Some of the columns selected will not be used directly in the model,
# but will be used to generate other features.
#
# Create variables specifying the features that will be used

# All categorical columns other than "loan_status" will be used as
# discrete features

discrete_features = list(set(cat_cols) - set(["loan_status"]))

# All numeric columns will be used as continuous features
continuous_features = list(float_cols + perc_cols)

## Ingestion
Ingest the data files from both sets, perform consistency checks, and prepare one single file for each set

In [34]:
def ingest_files(directory):
    '''
    This function will ingest every file in the specified directory
    into a pandas dataframe. It will return a dictionary containing
    these dataframes, keyed by the file name.
    
    We assume the directory contains files directly downloaded from
    the link given in the handout, and *only* those files. Thus, we 
    assume the files are zipped (pd.read_csv can read zipped files) 
    and we assume the first line in each file needs to be skipped.
    
    Note that each file will be read *without* formatting
    '''
    
    # If the directory has no trailing slash, add one
    if directory[-1] != "/":
        directory =  directory + "/"
    all_files = []
    for file in os.listdir(directory):
        if file.endswith(".zip"):
            all_files.append(file)
    output = {}
    
    print("Directory " + directory + " has " + str(len(all_files)) + " files:")
    for i in all_files:
        print("    Reading file " + i)
        output[i] = pd.read_csv(directory + i, dtype='str', header=1) # read each with dtype='str' and skip_rows =1
        
        # Some of the files have "summary" lines that, for example
        # read "Total number of loans number in Policy 1: ....."
        # To remove those lines, find any lines with non-integer IDs
        # and remove them
        l = output[i]['id'].tolist()
        invalid_rows = [x for x, y in enumerate(l) if not is_integer(y)]# mask rows that have non-integer IDs. Use is_integer method
        if len(invalid_rows) != 0:
            print("Found " + str(len(invalid_rows)) + " invalid rows which were removed")
            output[i].drop(output[i].index[invalid_rows])
    
    return output # return dictionary of dataframe

In [35]:
# Ingest the set of files we downloaded using the defined method "ingest_files"
files_cs = ingest_files(dir_cs) # dictioary of (filename, dataframe) as (key, value)

Directory /Users/vicky/Documents/GitHub/LendingClubCase/1805_download/ has 12 files:
    Reading file LoanStats_securev1_2016Q3.csv.zip
Found 2 invalid rows which were removed
    Reading file LoanStats_securev1_2016Q2.csv.zip
Found 2 invalid rows which were removed
    Reading file LoanStats3a_securev1.csv.zip
Found 3 invalid rows which were removed
    Reading file LoanStats_securev1_2017Q1.csv.zip
Found 2 invalid rows which were removed
    Reading file LoanStats3d_securev1.csv.zip
Found 2 invalid rows which were removed
    Reading file LoanStats_securev1_2017Q3.csv.zip
Found 2 invalid rows which were removed
    Reading file LoanStats_securev1_2017Q2.csv.zip
Found 2 invalid rows which were removed
    Reading file LoanStats_securev1_2016Q1.csv.zip
Found 2 invalid rows which were removed
    Reading file LoanStats_securev1_2016Q4.csv.zip
Found 2 invalid rows which were removed
    Reading file LoanStats3c_securev1.csv.zip
Found 2 invalid rows which were removed
    Reading file Loa

### Combine the files

In [36]:
li = []
for key, value in files_cs.items():
    li.append(value)
data_cs = pd.concat(li, ignore_index=True) # combine "files_cs" into a pandas dataframe
              # resent index with drop = True

## Prepare Final Dataset

In [37]:
# Keep only the columns of interest from 'data_cs'
final_data = data_cs[data_cs.columns[data_cs.columns.isin(cols_to_pick)]]


In [38]:
print("Starting with " + str(len(final_data)) + " rows")

Starting with 1765451 rows


### Typecast the columns

In [39]:
# Remember that we read the data as string (without any formatting). 
# Now we would typecast the columns based on feature types which you found out in CS Phase 1

for i in float_cols:
    final_data[i] = final_data[i].astype('float') # typecast float columns

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [40]:
def clean_perc(x):
    if pd.isnull(x):
        return np.nan
    else:
        return float(x.strip()[:-1])

for i in perc_cols:
    final_data[i] = final_data[i].apply(clean_perc) # apply clean_perc to percentage columns

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [41]:
def clean_date(x):
    if pd.isnull(x):
        return None
    else:
        return datetime.datetime.strptime( x, "%b-%Y").date()

for i in date_cols:
    final_data[i] = final_data[i].apply(clean_date) # typecast date cloumns to datatime using clean_date

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [42]:
for i in cat_cols:
    final_data[i] = final_data[i].replace(np.nan, None, regex=True)
    final_data[i] = final_data[i].replace(r'\s+', None, regex=True,)# for categorical features if the value is null/empty set it to None

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


## Calculate returns for each loan

In [44]:
# Define the names of the four returns we'll be calculating as described in Q.6
# ret_PESS: Pessimistic return
# ret_OPT: Optimistic return
# ret_INTa, ret_INTb: Method3 at two differnt values of "i"
ret_cols = ["ret_PESS", "ret_OPT", "ret_INTa", "ret_INTb"]

In [45]:
# Remove all rows for loans that were paid back on the days they were issued
final_data['loan_length'] = (final_data.last_pymnt_d - final_data.issue_d) / np.timedelta64(1, 'M')
n_rows = len(final_data)

loan_length_greater_than_0 = final_data['loan_length'] != 0
final_data = final_data[loan_length_greater_than_0] # select rows where loan_length is not 0. 

print("Removed " + str(n_rows - len(final_data)) + " rows")

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


Removed 6218 rows


### M1-Pessimistic Method

In [48]:
final_data.columns

Index(['id', 'loan_amnt', 'funded_amnt', 'term', 'int_rate', 'installment',
       'grade', 'emp_length', 'home_ownership', 'annual_inc',
       'verification_status', 'issue_d', 'loan_status', 'purpose', 'dti',
       'delinq_2yrs', 'earliest_cr_line', 'fico_range_low', 'fico_range_high',
       'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_pymnt',
       'recoveries', 'last_pymnt_d', 'loan_length'],
      dtype='object')

In [74]:
# Calculate the return using a simple annualized profit margin
# Pessimistic definition (Handout 6a.) (M1)

final_data['term_num'] = final_data.term.str.extract('(\d+)',expand=False).astype(int) # length of loan in months

final_data['ret_PESS'] = ((final_data['total_pymnt'] - final_data['funded_amnt']) / 
                          final_data['funded_amnt']) * (12 / final_data['term_num'])

In [75]:
criteria1 = final_data['loan_status'] == 'Fully Paid'
criteria2 = final_data['loan_status'] == 'Charged Off'
final_data[criteria1 | criteria2].groupby('grade').mean()

,loan_amnt,funded_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,fico_range_low,fico_range_high,open_acc,pub_rec,revol_bal,revol_util,total_pymnt,recoveries,loan_length,term_num,ret_PESS
grade,,,,,,,,,,,,,,,,,,
A,13720.675783,13705.922743,7.226147,419.678318,87552.821959,15.334059,0.183057,727.115618,731.116134,11.819105,0.104012,17953.620162,39.671009,14416.940089,43.072409,21.552470,36.604167,0.016534
B,13099.002608,13087.092031,10.850455,403.002255,75614.074570,17.115259,0.309732,697.185753,701.185835,11.372827,0.209916,15938.189656,51.553367,13715.911024,103.314169,20.614325,38.700331,0.015532
C,13994.158359,13986.814639,14.067120,422.951086,72247.026682,18.559948,0.344667,687.684668,691.684703,11.457227,0.242380,15638.311975,55.996011,14190.983843,206.515938,19.037213,42.293356,0.005764
D,15100.479466,15092.466880,17.529922,464.363825,69926.527940,19.702807,0.358861,682.961538,686.961538,11.509335,0.245029,15349.312189,58.734568,14855.219497,333.777717,18.600838,44.699509,-0.000147
E,17692.437972,17667.081249,20.716908,525.910621,72458.833358,20.388148,0.355929,681.687331,685.687331,11.824565,0.239950,16334.950874,59.639968,16751.342627,530.525608,17.768241,50.878259,-0.009972
F,19107.094982,19073.208707,24.453986,580.716116,73742.936752,20.502070,0.363597,679.425640,683.425679,11.978212,0.238918,16058.178932,60.549683,17517.538379,722.631428,16.897006,54.440587,-0.015333
G,20701.586069,20680.904318,27.082773,654.669567,77288.174763,20.696986,0.379474,677.766193,681.766193,12.231884,0.243567,16806.380804,59.994163,18296.768385,824.464336,14.691292,56.362023,-0.027774


### M2-Optimistic Method

In [ ]:
# Assuming that if a loan gives a positive return, we can
# immediately find a similar loan to invest in; if the loan
# takes a loss, we use M1-pessimistic to compute the return

final_data['ret_OPT'] = ...

final_data.loc[final_data.ret_OPT < 0,'ret_OPT'] = ...

### Method 3

In [ ]:
def ret_method_3(T, i):
    '''
    Given an investment time horizon (in months) and re-investment
    interest rate, calculate the return of each loan
    '''
    
    # Assuming that the total amount paid back was paid at equal
    # intervals during the duration of the loan, calculate the
    # size of each of these installment
    actual_installment = (final_data.total_pymnt - final_data.recoveries) / ...

    # Assuming the amount is immediately re-invested at the prime
    # rate, find the total amount of money we'll have by the end
    # of the loan
    cash_by_end_of_loan = actual_installment * ... # compute the quantity given in [] in eq.2.3 of handout
    
    cash_by_end_of_loan = cash_by_end_of_loan + final_data.recoveries
    
    # Assuming that cash is then re-invested at the prime rate,
    # with monthly re-investment, until T months from the start
    # of the loan
    remaining_months = T - final_data['loan_length']
    final_return = cash_by_end_of_loan * ... 

    # Find the percentage return
    ret_val = (12/T) * ...
    return ret_val

In [ ]:
final_data['ret_INTa'] = ... # call ret_method_3 with T=60, i=0.002
final_data['ret_INTb'] = ... # call ret_method_3 with T=60, i=0.005

### Visualize the variables

In [ ]:
def visualize_float_columns():
    '''
    This function visualizes Box-and-whisker plots for continuous variables
    '''
    
    # FLoat columns
    for i in float_cols + perc_cols + ret_cols:
        seaborn.boxplot(final_data[i])

        # Print the three highest values
        highest_vals = ... # get 3 highest values
        
        smallest_val = min(final_data[i])
        
        plt.text(smallest_val, -0.3, highest_vals[0])
        plt.text(smallest_val, -0.2, highest_vals[1])
        plt.text(smallest_val, -0.1, highest_vals[2])

        plt.show()

In [ ]:
def visualize_cat_columns():
    '''
    Lists the distinct values for categorical columns
    '''
    # Categorical columns 
    for i in cat_cols:
        ... # print field name
        ... # print number of distinct values
        ... # for each distinct value print the number of occurances
        print("")
        print("")

In [ ]:
def visualize_date_columns():
    '''
    This function visualizes a timeline density for dates
    '''
    
    # Date columns
    for i in date_cols:
        final_data[final_data[i].isnull() == False][i].apply(lambda x : str(x.year) +
                                                "-" + str(x.month)).value_counts(ascending = True).plot()
        plt.title(i + " (" + str(final_data[i].isnull().sum()) + " null values)")
        plt.show()

In [ ]:
# visualize continuous features
...

# visulaize categorical features
...

# visualize date columns
...

### Handle outliers

In [ ]:
# There are quite a few outliers. 
# Please identify top-k (decide this based on the visualization) features where outliers are most obvious
n_rows = len(final_data)

final_data = ... # remove outliers based 1st obvious feature
final_data = ... # remove outliers based 2nd obvious feature
...
final_data = ... # remove outliers based kth obvious feature

print("Removed " + str(n_rows - len(final_data)) + " rows")

In [ ]:
# Remove all loans that are still current
n_rows = len(final_data)

final_data = ...

print("Removed " + str(n_rows - len(final_data)) + " rows")

In [ ]:
# Only include loans isssued since 2010
n_rows = len(final_data)

final_data = ...

print("Removed " + str(n_rows - len(final_data)) + " rows")

### Drop null values

In [ ]:
# Deal with null values. We allow cateogrical variables to be null
# OTHER than grade, which is a particularly important categorical.
# All non-categorical variables must be non-null, and we drop
# rows that do not meet this requirement

required_cols = set(cols_to_pick) - set(cat_cols) - set(["id"])
required_cols.add("grade")

n_rows = len(final_data)

... # drop rows that contain null based only on "required_cols"

print("Removed " + str(n_rows - len(final_data)) + " rows")

### Visualize clean data

In [ ]:
# Visualize the data again after cleaning
...
...
...

In [ ]:
# Visualize the feature correlations
... # use sns scatter or pairplot
...

In [ ]:
# Visualize relation between loan status and features
... # sns pairplot or scatter plot. Refer to recitations
...
...

What do you observe after removing the outliers?

### Data Exploration
Solution to Q.7 from the handout

In [ ]:
# Find the percentage of loans by grade, the default by grade,
# and the return of each grade
perc_by_grade = (final_data.grade.value_counts()*100/len(final_data)).sort_index()

default_by_grade = final_data.groupby("grade").apply(lambda x : (x.loan_status != "Fully Paid").sum()*100/len(x) )
ret_by_grade_OPT = ... # average return for M2-Optimistic for each loan grade
ret_by_grade_PESS = ... # average return for M1-Pessimistic for each loan grade
ret_by_grade_INTa = ... # average return for M3
ret_by_grade_INTb = ... # average return for M3
int_rate_by_grade = ... # average interest rate for each grade

combined = pd.DataFrame(perc_by_grade)
combined.columns = ['perc_of_loans']
combined['perc_default'] = default_by_grade
combined['avg_int_rate'] = int_rate_by_grade
combined['return_OPT'] = ret_by_grade_OPT
combined['return_PESS'] = ret_by_grade_PESS
combined['return_INTa'] = ret_by_grade_INTa
combined['return_INTb'] = ret_by_grade_INTb

combined

Based on the output of previous cell, write down your answers to Q.7 from the handout.

### Save a Pickle

In [ ]:
# Remove the "total_pymnt" and "recoveries" from the list of continuous features
continuous_features = ...

Why did we remove `total_pymt` and `recoveries` from the data for the task of predicting whether to give loan or not, although these are highly predictive features?

In [ ]:
# save the prepared data for modeling in next Phase.
pickle.dump( [final_data, discrete_features, continuous_features, ret_cols], open(pickle_file, "wb") )